In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
import numpy as np
import json
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split

In [2]:
# Model definition
def Unet(inputshape = (640,640,3), num_classes=2):
    inputs = Input(inputshape)
    
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p3)
    
    # u6 = (UpSampling2D((2, 2))(c5))
    # u6 = Concatenate()([u6, c4])
    # c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    
    u7 = (UpSampling2D((2, 2))(c5))
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    
    u8 = (UpSampling2D((2, 2))(c7))
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    
    u9 = (UpSampling2D((2, 2))(c8))
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)
    
    return Model(inputs, outputs)

# Function to load the dataset (also implicitly performs data processing)
def load_dataset(image_dir, mask_dir, img_size=(640, 640)):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]
    mask_paths = [os.path.join(mask_dir, mask) for mask in os.listdir(mask_dir)]

    images = [tf.image.decode_png(tf.io.read_file(img)) for img in image_paths]
    masks = [tf.image.decode_png(tf.io.read_file(mask)) for mask in mask_paths]

    images = tf.stack(images)
    masks = tf.stack(masks)

    return data_processing(images, masks, img_size)
def data_processing (images, masks, image_size=(640,640)):
    images = tf.image.resize(images,image_size) / 255.0
    masks = tf.image.resize(masks,image_size) / 255.0
    masks = tf.cast(masks, tf.int32)
    masks = tf.one_hot(masks, depth=2)
    return np.array(images), np.array(masks)

# Function to generate segmentation masks from COCO JSON annotations
def generate_segmentation_masks(json_file, output_dir, image_size=(640, 640)):
    
    # Load the JSON file
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Unpack the dimensions
    height, width = image_size
    
    # Iterate over each image in the annotations
    for image_info in annotations['images']:
        image_id = image_info['id']
        image_filename = image_info['file_name']
        
        # Create a blank mask
        mask = np.zeros((height, width), dtype=np.uint8)
        
        # Get annotations for the current image
        for annotation in annotations['annotations']:
            if annotation['image_id'] == image_id:
                if 'segmentation' in annotation and annotation['segmentation']:
                    # Assuming polygon segmentation
                    polygon = annotation['segmentation'][0]
                    polygon_points = [
                        (polygon[i], polygon[i + 1]) for i in range(0, len(polygon), 2)
                    ]
                    
                    # Draw polygon on the mask
                    img = Image.fromarray(mask)
                    draw = ImageDraw.Draw(img)
                    draw.polygon(polygon_points, outline=1, fill=1)
                    mask = np.array(img)
        
        # Save the mask as a PNG image
        mask_filename = os.path.join(
            output_dir, f"{os.path.splitext(image_filename)[0]}_mask.png"
        )
        Image.fromarray(mask * 255).save(mask_filename)
    
    print(f"Masks saved to {output_dir}")

In [3]:
model = Unet(inputshape=(640,640,3), num_classes=2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

2025-01-26 13:22:36.685835: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-01-26 13:22:36.685867: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-26 13:22:36.685871: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-26 13:22:36.686295: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-26 13:22:36.686312: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 640, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 640, 640,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 320, 320,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 320, 320,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 160, 160,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 160, 160,  │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 80, 80,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 80, 80,    │  2,360,320 │ max_pooling2d_2[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 160, 160,  │          0 │ conv2d_3[0][0]    │
│ (UpSampling2D)      │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160, 160,  │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 1280)             │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 160, 160,  │  2,949,376 │ concatenate[0][0] │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 320, 320,  │          0 │ conv2d_4[0][0]    │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 320, 320,  │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 384)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 320, 320,  │    442,496 │ concatenate_1[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_2     │ (None, 640, 640,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 640, 640,  │          0 │ up_sampling2d_2[… │
│ (Concatenate)       │ 192)              │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 640, 640,  │    110,656 │ concatenate_2[0]

 Total params: 6,233,794 (23.78 MB)

 Trainable params: 6,233,794 (23.78 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
image_fol = '/Volumes/External SW/U-Net WT/dataset/image'
mask_fol = '/Volumes/External SW/U-Net WT/dataset/masks'

images, masks = load_dataset(image_dir=image_fol, mask_dir=mask_fol, img_size=(640, 640))

In [5]:
train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.2, random_state=42
)

In [6]:
train_images = tf.convert_to_tensor(train_images)
val_images = tf.convert_to_tensor(val_images)
train_masks = tf.convert_to_tensor(train_masks)
val_masks = tf.convert_to_tensor(val_masks)

In [7]:
print(f"Train Images Shape: {train_images.shape}")
print(f"Train Masks Shape: {train_masks.shape}")
print(f"Validation Images Shape: {val_images.shape}")
print(f"Validation Masks Shape: {val_masks.shape}")

Train Images Shape: (405, 640, 640, 3)
Train Masks Shape: (405, 640, 640, 1, 2)
Validation Images Shape: (102, 640, 640, 3)
Validation Masks Shape: (102, 640, 640, 1, 2)


In [8]:
train_masks = tf.squeeze(train_masks, axis=-2)  
val_masks = tf.squeeze(val_masks, axis=-2)      

In [9]:
print(f"Train Images Shape: {train_images.shape}")
print(f"Train Masks Shape: {train_masks.shape}")
print(f"Validation Images Shape: {val_images.shape}")
print(f"Validation Masks Shape: {val_masks.shape}")

Train Images Shape: (405, 640, 640, 3)
Train Masks Shape: (405, 640, 640, 2)
Validation Images Shape: (102, 640, 640, 3)
Validation Masks Shape: (102, 640, 640, 2)


In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks)).batch(16)

In [11]:
train_masks = 0
val_masks = 0

In [ ]:
a = model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10


2025-01-26 13:23:50.643148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
